<a href="https://colab.research.google.com/github/JGonzalez249/RandomNovelGen/blob/main/Copy_of_Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: October 17th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [1]:
!pip install -q gpt-2-simple
!pip install tracery
import gpt_2_simple as gpt2
import tracery
from datetime import datetime
from google.colab import files

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tracery: filename=tracery-0.1.1-py3-none-any.whl size=7697 sha256=e7ffdadd1a8faf4cd59eabe5ddf38e8f736108c69684f70878672acdeafdbacc
  Stored in directory: /root/.cache/pip/wheels/2c/55/dd/ca1cff9fcfa0968ca0610769213fc9e907c88eb2c0164726d4
Successfully built tracery


## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Wed Feb 15 19:56:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    35W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 464Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.06Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 543Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:18, 27.4Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 585Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 6.86Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.67Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [2]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "revisedData.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

use_memory_saving_gradients = True
only_train_transformer_layers = True
accumulate_gradients = 1

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=2500,
              restore_from='fresh',
              run_name='run1',
              print_every=100,
              sample_every=200,
              save_every=500,
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


dataset has 397580 tokens
Training...
[100 | 253.27] loss=2.99 avg=2.99
[200 | 498.45] loss=2.64 avg=2.82
======== SAMPLE 1 ========
, which will have a greater degree of importance if a man is so inclined as to consider himself as acting instinctively by the action of his hands and feet. He must be a type of man of the highest importance which man has ever produced; perhaps he
will feel himself to have the highest degree of autonomy and independence--in
arguments, in deduction from the facts of circumstances, in all sorts of perplexities
and prejudices: he will therefore feel himself under the influence of the strongest instincts
of his
world; in the most obvious sense, he will feel himself to be a type of the
greatest class of men which exists on earth which has not only produced man
from the first, but has hitherto been regarded as a species of man from the
place of his origin. But what did man make of the first man? In some sense, he
made the first man--he was himself the Great Old

Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========


For a moment there was a stir that seemed to shutters on
down the one end of the lane, as well as a couple of hundred yards or
MORE down the lane. The police, who had had enough of the pony-pulling,
turned about and surrounded the pony-pulling boys, who in all the
time that I heard, had gone noiselessly and in nothing but a waggon,
and been a long-suffering and quietly mocking race to these, the most
inclined and loathsome faces. And this came to my uncle’s rescue
when, out of sheer horror, he fled in this crowd of about ten yards
or more. They arrested him, took him to aaihop in Bushey Park, bore him
through upon some wicker devices and even into the forest, whipped him with
barbaric blows, and held him in ahole for some time. Then, out of
crowd, to let himself be swept away and to let his horrible repose
crouch upon the park bench forever; there was nothing but confusion and
frightfulness upon that sombre evening’s heat and frost.

By midnight my uncle’s 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [3]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [4]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-2500


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [10]:
gpt2.generate(sess, run_name='run1')

I could see her
shaking her head. “Good-evening,” she said.

For a moment she had staggered and was swaying her headlong from
fireplace to place. I could only push her to the floor and not hear
her whim. I then shook my head.

“Do you not wish to disturb her, Doctor?” I said.

She nodded. “Then you can go upstairs and make her wash her hands.”

She walked slowly off, and I heard her faint breathing under the
heavy burdens which her heavy burden placed upon me.

“Her room, I think, is the one which she has agreed to stay in.”

“Very good. Now I will try and get the books which she has
opened out of the box which she has left behind me. One of them is
called ‘The Leopard.’ It is an epilogue—one may say the beginning—of a
long story; it is a pity that you care to read it. Here is the box
which she has left behind me.”

“Excellent. Now, then! What will you tell us?”

“It is all over now, and yet I do not see a sequel.”

“I see that there is at least one possible conclusion.”

“Some little 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [14]:
from tensorflow._api.v2.math import top_k
prefix = "[story: #story#]"

rules = {
    "story": ["#protagonist# was #verb.ing# #place#. #event#. #conflict#. #complication#. #goal#"],
    "name": ["Adam", "Alice", "Alex", "Anna", "Ben", "Beth", "Bob", "Brianna", "Carl", "Cathy", "Chris", "Clara", "Dan", "Dana", "Dave", "Diana", "Ed", "Ella", "Eric", "Emma", "Frank", "Fiona", "Gary", "Grace", "Harry", "Hannah", "Ian", "Isabel", "Jack", "Jade", "James", "Jenna", "John", "Julia", "Kevin", "Kate", "Leo", "Lily", "Mark", "Mia", "Matt", "Molly", "Nick", "Nora", "Owen", "Olivia", "Paul", "Penny", "Peter", "Rachel", "Ray", "Rose", "Ryan", "Ruby", "Sam", "Sara", "Scott", "Sophia", "Tom", "Tara", "Tim", "Tina", "Will", "Wendy", "Zack", "Zoe", "Aaron", "Abby", "Andy", "Amy", "Blake", "Bella", "Cody", "Chloe", "Drew", "Daisy", "Evan", "Eva", "Finn", "Faith", "Greg", "Gina", "Henry", "Hailey", "Isaac", "Ivy", "Jesse", "Jill", "Kyle", "Kayla", "Luke", "Leah", "Max", "Mila", "Nate", "Nina", "Oscar", "Opal", "Quinn", "Quinn", "Rory", "Renee", "Seth", "Sadie", "Troy", "Tracy", "Victor", "Vera", "Wyatt", "Willa", "Xavier", "Xena", "Yuri", "Yara", "Zane", "Zara"],
    "protagonist": ["#name.capitalize# the #occupation#", "a #adjective# #occupation#", "a #occupation# named #name.capitalize#"],
    "noun": ["apple", "ball", "cat", "dog", "egg", "fish", "girl", "hat", "ice", "jam", "key", "lamp", "map", "net", "owl", "pen", "queen", "rat", "sun", "toy", "umbrella", "vase", "web", "x-ray", "yarn", "zebra", "airplane", "banana", "cake", "door", "elephant", "fire", "grass", "house", "jacket", "kite", "lemon", "moon", "nose", "orange", "pig", "quilt", "rain", "star", "tree", "violin", "window", "xylophone", "yogurt", "zipper", "ant", "book", "car", "desk", "eye", "flower", "guitar", "horse", "ink", "jar", "knife", "lion", "milk", "nut", "octopus", "phone", "quill", "rose", "shoe", "table", "van", "water", "yolk", "bag", "box", "chair", "duck", "ear", "fox", "iron", "juice", "kangaroo", "lock", "mouse", "nest", "pencil", "question", "ring", "sock", "train", "vest", "watch", "zeppelin"],
    "verb.s": ["walks", "runs", "jumps", "flies", "swims", "dances", "sings", "plays", "talks", "listens", "reads", "writes", "draws", "paints", "cooks", "eats", "drinks", "sleeps", "dreams", "laughs", "cries", "smiles", "frowns", "nods", "shakes", "claps", "waves", "hugs", "kisses", "loves", "hates", "likes", "dislikes", "wants", "needs", "has", "gives", "takes", "makes", "breaks", "fixes", "builds", "destroys", "creates", "learns", "teaches", "studies", "works", "rests", "relaxes", "helps", "hurts", "heals", "saves", "kills", "fights", "wins", "loses", "catches", "throws", "hits", "kicks", "pushes", "pulls", "lifts", "drops", "carries", "holds"],
    "verb.ing": ["seeing", "visiting", "exploring", "discovering", "learning", "reading", "writing", "painting", "singing", "dancing", "acting", "fighting", "hiding", "running", "flying", "swimming", "climbing", "jumping", "dreaming", "wishing", "hoping", "loving", "hating", "fearing", "trusting", "betraying", "helping", "hurting", "saving", "destroying"],
    "adjective": ["abandoned", "able", "absolute", "academic", "acceptable", "acclaimed", "accomplished", "accurate", "aching", "acidic", "acrobatic", "active", "actual", "adept", "admirable", "admired", "adolescent", "adorable", "adored", "advanced", "adventurous", "affectionate", "afraid", "aged", "aggravated", "aggressive", "agile", "agitated", "agonizing", "agreeable", "ajar", "alarmed", "alarming", "alert", "alienated", "alive", "all", "altruistic", "amazing", "ambitious", "ample", "amused", "amusing", "anchored", "ancient", "angelic", "angry", "anguished", "animated", "annual", "another", "antique", "anxious", "any", "apprehensive", "appropriate", "apt", "arctic", "arid", "aromatic", "artistic", "ashamed", "assured", "astonishing", "athletic", "attached", "attentive", "attractive", "austere", "authentic", "authorized", "automatic", "avaricious", "average", "aware", "awesome", "awful", "awkward", "babyish", "bad", "back", "baggy", "bare", "barren", "basic", "beautiful", "belated", "beloved", "beneficial", "better", "best", "bewitched", "big", "big-hearted", "biodegradable", "bite-sized", "bitter", "black", "black-and-white", "bland", "blank", "blaring", "bleak", "blind", "blissful", "blond", "blue", "blushing", "bogus", "boiling", "bold", "bony", "boring", "bossy", "both", "bouncy", "bountiful", "bowed", "brave", "breakable", "brief", "bright", "brilliant", "brisk", "broken", "bronze", "brown", "bruised", "bubbly", "bulky", "bumpy", "buoyant", "burdensome", "burly", "bustling", "busy", "buttery", "buzzing", "calculating", "calm", "candid", "canine", "capital", "carefree", "careful", "careless", "caring", "cautious", "cavernous", "celebrated", "charming", "cheap", "cheerful", "cheery", "chief", "chilly", "chubby", "circular", "classic", "clean", "clear", "clear-cut", "clever", "close", "closed", "cloudy", "clueless", "clumsy", "cluttered", "coarse", "cold", "colorful", "colorless", "colossal", "comfortable", "common", "compassionate", "competent", "complete", "complex", "complicated", "composed", "concerned", "concrete", "confused", "conscious", "considerate", "constant", "content", "conventional", "cooked", "cool", "cooperative", "coordinated", "corny", "corrupt", "costly", "courageous", "courteous", "crafty", "crazy", "creamy", "creative", "creepy", "criminal", "crisp", "critical", "crooked", "crowded", "cruel", "crushing", "cuddly", "cultivated", "cultured", "cumbersome", "curly", "curvy", "cute", "cylindrical", "damaged", "damp", "dangerous", "dapper", "daring", "darling", "dark", "dazzling", "dead", "deadly", "deafening", "dear", "dearest", "decent", "decimal", "decisive", "deep", "defenseless", "defensive", "defiant", "deficient", "definite", "definitive", "delayed", "delectable", "delicious", "delightful", "delirious", "demanding", "dense", "dental", "dependable", "dependent", "descriptive"],
    "occupation": ["accountant", "actor", "architect", "artist", "astronaut", "athlete", "baker", "banker", "barber", "biologist", "builder", "chef", "chemist", "clerk", "coach", "dentist", "designer", "doctor", "driver", "editor", "engineer", "farmer", "firefighter", "florist", "gardener", "journalist", "judge", "lawyer", "librarian", "mechanic", "musician", "nurse", "painter", "pharmacist", "photographer", "pilot", "plumber", "poet", "policeman", "politician", "programmer", "singer", "soldier", "student", "tailor", "teacher", "therapist", "translator", "veterinarian", "waiter", "writer", "yoga instructor", "zookeeper", "account executive", "animator", "archaeologist", "astronomer", "author", "bartender", "beautician", "broker", "bus driver", "carpenter", "cashier", "chiropractor", "comedian", "counselor", "detective", "diplomat", "director", "electrician", "entrepreneur", "fashion designer", "filmmaker", "flight attendant", "geologist", "graphic designer", "historian", "host", "illustrator", "instructor", "interior designer", "jeweler", "landscaper", "linguist", "magician", "manager", "model", "nanny", "optician", "paralegal", "philosopher", "physicist", "planner", "podcaster", "producer", "psychologist", "receptionist", "reporter", "salesperson", "scientist", "secretary", "social worker", "surgeon", "taxi driver", "technician", "tour guide", "trainer", "tutor", "volunteer", "web developer"],
    "place": ["a #adjective# #noun#", "a #noun# of #noun#", "a #noun# that #verb.s#"],
    "event": ["#protagonist# met #character#", "#protagonist# found #object#", "#protagonist# witnessed #phenomenon#", "#protagonist# received #message#", "#protagonist# heard #sound#", "#character# approached #protagonist#", "#character# gave #protagonist# #object#", "#character# told #protagonist# #message#", "#character# attacked #protagonist#", "#object# exploded", "#object# disappeared", "#object# transformed", "#phenomenon# occurred", "#phenomenon# stopped", "#message# arrived", "#sound# echoed"],
    "character": ["a #adjective# #noun#", "a #adjective# #occupation#", "a #occupation# named #name.capitalize#", "#name.capitalize# the #occupation#", "#name.capitalize# the #adjective# #noun#"],
    "object": ["a #adjective# #noun#", "a #noun# of #noun#", "a #noun# that #verb.s#"],
    "phenomenon": ["a #adjective# #noun#", "a #noun# of #noun#", "a #noun# that #verb.s#"],
    "message": ["a #adjective# #noun#", "a #noun# of #noun#", "a #noun# that #verb.s#"],
    "sound": ["a #adjective# #noun#", "a #noun# of #noun#", "a #noun# that #verb.s#"],
    "secret": ["#protagonist# was #verb.ing# #place#", "#protagonist# had #verb.ed# #character#", "#protagonist# knew #message#", "#protagonist# could #verb# #object#", "#protagonist# wanted to #verb# #character#"],
    "conflict": ["but #protagonist# had a problem: #problem#", "but #protagonist# faced a challenge: #challenge#", "but #protagonist# encountered a danger: #danger#", "but #protagonist# was not alone: #antagonist#"],
    "problem": ["#protagonist# was #verb.ing# #place#", "#protagonist# had #verb.ed# #character#", "#protagonist# knew #message#", "#protagonist# could #verb# #object#", "#protagonist# wanted to #verb# #character#"],
    "challenge": ["#protagonist# had to #verb# #object#", "#protagonist# had to #verb# #character#", "#protagonist"],
    "complication": ["a #noun# #adjective# hidden in an old #noun#", "a #conflict# that threatens to tear them apart", "a broken #noun# in the middle of nowhere", "a #secret# that could ruin everything", "a #problem# that wants them dead", "a #conflict# that defies the rules", "a missing #noun# that unlocks a #noun#", "a #problem# that tests their #noun#", "a #noun# #adjective# that holds a #noun#", "a #problem# that spreads rapidly", "a stolen #noun# that leads to a #noun#", "a #secret# that haunts them", "a #noun# #adjective# that grants a #noun#", "a #conflict# that challenges their #noun#", "a #noun# #adjective# that controls their #noun#", "a #secret# that attracts attention", "a #problem# that reveals a #noun#", "a #conflict# that could pay off", "a #noun# #adjective# that offers a #noun#", "a #secret# that exposes a #noun#"],
    "goal": ["to explore a new planet", "to invent a time machine", "to contact an alien civilization", "to save the world from a disaster", "to discover a cure for a deadly disease", "to escape from a dystopian society", "to hack a powerful corporation", "to survive a zombie apocalypse", "to create artificial intelligence", "to join a rebel faction", "to uncover a conspiracy", "to colonize a new world", "to find a lost artifact", "to travel to another dimension", "to reverse a genetic mutation", "to steal a spaceship", "to defy the laws of physics", "to fight a robot army", "to solve a mystery", "to win a virtual reality game"],
}
gpt2.generate(sess,
              run_name='run1',
              length=1000,
              temperature=1,
              prefix=prefix,
              top_k=10,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

[story: #story#] (a violation of both
the law and the good sense of his audience), the highest happiness of an
artist
to exhibit one's emotions in a form which he is familiar with and
which he enjoys even among his friends;--who could refrain therefrom
from this great art and this sublime exultation of his? And above all,
do not let this be repugnant to the honour and gratitude which
the great man feels to be heeded by the use of his organs (an
adornment which appears to be agreeable to his taste, but which actually
flagged and diminished in value, like a cloak which has worn away from
your shoulders?), to the consciousness that he has no clear idea what HE'S DOING, and
to the ominous suspicion which may have beenhes him out, in effect,
in the open air of his new-found happiness: a man who has "heavily"
influenced a step or two of his own doctrine, which has developed into
a philosophy (as has been suggested to me by your correspondent, your
friend), has still too often dug deeply, and

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.